In [178]:
import json
import pandas as pd
from development.datasets.OsdgDataset import load_osdg_data
from development.datasets.uclmodules_dataset import load_uclmodules_data
from development.datasets.videscription_dataset import load_videscription_data
from development.train_model import fine_tune_transformer
from development.models.Bert import Bert
from development.models.RobertaNER import RobertaNER
from development.pipelines import full_pipe
from development.scrape.RelxScraper import RelxScraper
from development.utils import parse_sdg_label

In [112]:
%load_ext autoreload
%autoreload 2
pd.set_option('max_colwidth', None)

In [168]:
with open('config.json', 'r') as file:
    CONFIG = json.load(file)
    dev_config = CONFIG['development']


In [361]:
osdg_data = load_osdg_data(dev_config['osdg_data_path'], training=False, filter_agreement=False)

In [367]:
osdg_data.groupby(['sdg']).count()['text']

sdg
1     2734
2     2457
3     2689
4     3740
5     4338
6     2815
7     3048
8     1509
9     2105
10    2032
11    2277
12    1108
13    2102
14    1141
15    2143
16    5451
Name: text, dtype: int64

In [357]:
osdg_data = load_osdg_data(dev_config['osdg_data_path'], training=True, filter_agreement=False)

In [332]:
ucl_data = load_uclmodules_data(dev_config['uclmodules_data_path'])

In [5]:
videscription_data = load_videscription_data(
    dev_config['videscription_data_path']
)

In [ ]:
bert = Bert()

In [ ]:
result = fine_tune_transformer(
    bert.model,
    bert.tokenizer,
    bert.tokenizer_args,
    data=osdg_data,
    dev_config=dev_config
)

In [169]:
bert = Bert('./development/weights/Bert/checkpoint-3000/')

In [ ]:
cls_report = bert.evaluate(osdg_data['test'])
print(cls_report)

In [48]:
ner_model = RobertaNER()

In [27]:
ner_model.print_entities(osdg_data['train'][0][14])

ORG: The World Resources Institute
ORG: World Resources Institute


In [76]:
df = full_pipe(bert, ner_model, texts=[osdg_data['train'][0][7]])

,Text,SDG,Entities,Sentiment
0,"This is why the Sustainable Development of Protected Areas System of Ethiopia was set up, with support from the Global Environment Fund and UNDP. The project is spearheading a suite of interventions, focusing on the national system in terms of capacity building and training, and integrating the protected area system into mainstream development. Since the initiation of the project in 2008, valuation exercises have found that the main value of protected areas is in the environmental services that they provide to poor rural communities, many of which are food-insecure, protected areas were incorporated into the Ethiopia Poverty Strategy, and the legal boundaries of the protected area system were strengthened by supporting the demarcation and gazettement of four areas through a highly consultative process (UNDP, n.d.).",Climate Action,ORG: the Sustainable Development of Protected Areas System of Ethiopia - ORG: the Global Environment Fund - ORG: UNDP - ORG: UNDP -,NULL


In [138]:
relx_scraper = RelxScraper()
relx_scraper.scrape_data(start=0)
relx_scraper.save_as_csv('./data/relx_data.csv')

In [262]:
predictions = bert.predict_batch(relx_scraper.data['abstract'].values)

In [349]:
ucl_data[1][11]

[16, 4]

In [369]:
bert.evaluate_multilabel(ucl_data, add_one=True)

0.3105493133583021